<a href="https://colab.research.google.com/github/ldm0612/Datahouse-Consulting-Compatibility-Predictor/blob/main/Datahouse_Compatibility_Predictor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [18]:
import numpy as np
import json

1.**Read in sample input file.**

In [ ]:
with open(input_file.json, 'r') as json_file:
    data = json.load(json_file)

In [39]:
jason_file = '''
{
  "team": [
    {
      "name": "Eddie",
      "attributes": {
        "Endurance": 9,
        "Intelligence": 6,
        "SpicyFoodTolerance": 1,
        "Leadership": 7,
        "Strength": 8,
        "Extroversion": 3
      }
    },
    {
      "name": "Tom",
      "attributes": {
        "Endurance": 9,
        "Intelligence": 6,
        "SpicyFoodTolerance": 1,
        "Leadership": 7,
        "Strength": 8,
        "Extroversion": 3
      }
    },
    {
      "name": "Will",
      "attributes": {
        "Endurance": 7,
        "Intelligence": 2,
        "SpicyFoodTolerance": 7,
        "Leadership": 8,
        "Strength": 2,
        "Extroversion": 9
      }
    },
    {
      "name": "Mike",
      "attributes": {
        "Endurance": 8,
        "Intelligence": 9,
        "SpicyFoodTolerance": 9,
        "Leadership": 7,
        "Strength": 4,
        "Extroversion": 6
      }
    }
  ],
  "applicants": [
    {
      "name": "John",
      "attributes": {
        "Endurance": 5,
        "Intelligence": 5,
        "SpicyFoodTolerance": 5,
        "Leadership": 5,
        "Strength": 5,
        "Extroversion": 5
      }
    },
    {
      "name": "Jane",
      "attributes": {
        "Endurance": 9,
        "Intelligence": 5,
        "SpicyFoodTolerance": 3,
        "Leadership": 6,
        "Strength": 5,
        "Extroversion": 2
      }
    },
    {
      "name": "Gabe",
      "attributes": {
        "Endurance": 3,
        "Intelligence": 1,
        "SpicyFoodTolerance": 10,
        "Leadership": 10,
        "Strength": 5,
        "Extroversion": 7
      }
    },
    {
      "name": "Joe",
      "attributes": {
        "Endurance": 1,
        "Intelligence": 2,
        "SpicyFoodTolerance": 1,
        "Leadership": 1,
        "Strength": 10,
        "Extroversion": 9
      }
    }
  ]
}
'''

In [40]:
data = json.loads(jason_file)
team_members = data['team']
applicants = data['applicants']

In [42]:
team_members

[{'name': 'Eddie',
  'attributes': {'Endurance': 9,
   'Intelligence': 6,
   'SpicyFoodTolerance': 1,
   'Leadership': 7,
   'Strength': 8,
   'Extroversion': 3}},
 {'name': 'Tom',
  'attributes': {'Endurance': 9,
   'Intelligence': 6,
   'SpicyFoodTolerance': 1,
   'Leadership': 7,
   'Strength': 8,
   'Extroversion': 3}},
 {'name': 'Will',
  'attributes': {'Endurance': 7,
   'Intelligence': 2,
   'SpicyFoodTolerance': 7,
   'Leadership': 8,
   'Strength': 2,
   'Extroversion': 9}},
 {'name': 'Mike',
  'attributes': {'Endurance': 8,
   'Intelligence': 9,
   'SpicyFoodTolerance': 9,
   'Leadership': 7,
   'Strength': 4,
   'Extroversion': 6}}]

In [43]:
applicants

[{'name': 'John',
  'attributes': {'Endurance': 5,
   'Intelligence': 5,
   'SpicyFoodTolerance': 5,
   'Leadership': 5,
   'Strength': 5,
   'Extroversion': 5}},
 {'name': 'Jane',
  'attributes': {'Endurance': 9,
   'Intelligence': 5,
   'SpicyFoodTolerance': 3,
   'Leadership': 6,
   'Strength': 5,
   'Extroversion': 2}},
 {'name': 'Gabe',
  'attributes': {'Endurance': 3,
   'Intelligence': 1,
   'SpicyFoodTolerance': 10,
   'Leadership': 10,
   'Strength': 5,
   'Extroversion': 7}},
 {'name': 'Joe',
  'attributes': {'Endurance': 1,
   'Intelligence': 2,
   'SpicyFoodTolerance': 1,
   'Leadership': 1,
   'Strength': 10,
   'Extroversion': 9}}]

2. **Calculate team average for each attribute.**

In [44]:
# Create a list that stores the attribute scores for all team members
team_attributes= list(member["attributes"] for member in team_members )

# Create the list that contain the names of the attributes
attributes_list=list(team_attributes[1].keys())

In [45]:
attribute_mean_dict={key: 0 for key in attributes_list}

# Sum up the scores stored in team_attributes previously
for member in team_attributes:
  for attribute, score in member.items():
    attribute_mean_dict[attribute] += score

# Divide the score by the number of team members
for attribute, score in attribute_mean_dict.items():
  attribute_mean_dict[attribute] = score / len(team_attributes)

attribute_mean_dict

{'Endurance': 8.25,
 'Intelligence': 5.75,
 'SpicyFoodTolerance': 4.5,
 'Leadership': 7.25,
 'Strength': 5.5,
 'Extroversion': 5.25}

3. **Check for polarization for each attribute.**

In [46]:
# Create a dictionary that stores the categories of each attribute, including:
#   1. Is this attribute we value the most or is it just good to have
#   2. Do we want to find new applicants that are similar or diverse in this attribute to our team members?
#   3. Are our current team members polarized in this attribute (False by default)
attribute_cat={
  'Endurance': ["Value the most", "Similar", False],
  'Intelligence': ["Value the most", "Similar", False],
  'SpicyFoodTolerance': ["Good to have", "Similar", False],
  'Leadership': ["Value the most", "Diverse", False],
  'Strength': ["Good to have", "Diverse", False],
  'Extroversion': ["Good to have", "Diverse", False]
  }

In [47]:
# This function checks whether our current team members are polarized in an attribute

def check_polar(attributes_cat):

  # If the difference in mean scores is more than 4, than we consider our team to be polarized in this attribute
  polarization_threshold=4

  for attribute in attributes_list:
    # Extract all the score for the attribute from our members
    team_score=[member[attribute] for member in team_attributes]

    # Rank the scores from high to low
    sorted_team_score=sorted(team_score, reverse=True)

    # Find the midpoint score, here we used the floor division
    midpoint = len(sorted_team_score) // 2

    # Finding the mean for the higher half and the lower half
    first_half_mean = np.mean(sorted_team_score[:midpoint])
    second_half_mean = np.mean(sorted_team_score[midpoint:])

    # If the difference between the two means is greater than our set threshold, we mark the attribute as polarized.
    if first_half_mean-second_half_mean > polarization_threshold:
      attributes_cat[attribute][2]=True

  return attributes_cat

In [49]:
attribute_cat=check_polar(attribute_cat)
attribute_cat

{'Endurance': ['Value the most', 'Similar', False],
 'Intelligence': ['Value the most', 'Similar', False],
 'SpicyFoodTolerance': ['Good to have', 'Similar', True],
 'Leadership': ['Value the most', 'Diverse', False],
 'Strength': ['Good to have', 'Diverse', True],
 'Extroversion': ['Good to have', 'Diverse', True]}

4. **Computing the final score for each applicant**

In [78]:
# Here, we use a simple linear interpolation to convert the raw score to the final score
# Our input raw score range from 0 to 10, the raw score comes from the difference between the applicant's score and the team's mean score
# How the output score is generated by the raw score is based on the category of the specific attribute.
# Our output score for "Value the most" attribute range from 0-0.8; output score for "Good to have" attributes range from 0-0.2
def linear_scaling(x, v_or_g, s_or_d, polarized):
    x1 = 0
    x2 = 10
    if v_or_g == "Value the most":
       y1= 0.7
    elif v_or_g == "Good to have":
      y1 = 0.3
    y2 = 0

    if (s_or_d == "Similar" and polarized == False) or (s_or_d == "Diverse" and polarized == True):
      score = y1+((y2-y1)/(x2-x1))*(x-x1)
    elif (s_or_d == "Similar" and polarized == True) or (s_or_d == "Diverse" and polarized == False):
      score = y1-(y1+((y2-y1)/(x2-x1))*(x-x1))
    else:
      print("Error")

    return round(score, 1)

In [79]:
applicants

[{'name': 'John',
  'attributes': {'Endurance': 5,
   'Intelligence': 5,
   'SpicyFoodTolerance': 5,
   'Leadership': 5,
   'Strength': 5,
   'Extroversion': 5}},
 {'name': 'Jane',
  'attributes': {'Endurance': 9,
   'Intelligence': 5,
   'SpicyFoodTolerance': 3,
   'Leadership': 6,
   'Strength': 5,
   'Extroversion': 2}},
 {'name': 'Gabe',
  'attributes': {'Endurance': 3,
   'Intelligence': 1,
   'SpicyFoodTolerance': 10,
   'Leadership': 10,
   'Strength': 5,
   'Extroversion': 7}},
 {'name': 'Joe',
  'attributes': {'Endurance': 1,
   'Intelligence': 2,
   'SpicyFoodTolerance': 1,
   'Leadership': 1,
   'Strength': 10,
   'Extroversion': 9}}]

In [86]:
scoredApplicants = []

for applicant in applicants:
  applicant_attributes = applicant["attributes"]

  # Initialize two list to store the scores for "Value the most" attributes and "Good to have" attributes separately
  v_scores=[]
  g_scores=[]

  for attribute, applicant_score in applicant_attributes.items():
    cat=attribute_cat[attribute]
    diff_score=abs(applicant_score-attribute_mean_dict[attribute])
    if cat[1]== "Similar":
      # Sum up the scores generated by our linear interpolation
      v_scores.append(linear_scaling(x=diff_score, v_or_g=cat[0], s_or_d=cat[1], polarized=cat[2]))
    elif cat[1]== "Diverse":
      g_scores.append(linear_scaling(x=diff_score, v_or_g=cat[0], s_or_d=cat[1], polarized=cat[2]))
    else:
      print("Error")
      break

  # The final score is the sum of the mean "Value the most" score and the mean "Good to have" score.
  # If the applicant have perfect scores in both fields, then their final score will be 0.8+0.2=1.
  final_score= round(np.mean(v_scores)+np.mean(g_scores), 1)
  scoredApplicants.append({"name" : applicant["name"], "score": final_score})


In [84]:
scoredApplicants

[{'name': 'John', 'score': 0.6},
 {'name': 'Jane', 'score': 0.6},
 {'name': 'Gabe', 'score': 0.5},
 {'name': 'Joe', 'score': 0.5}]

In [ ]:
output_dict={"scoredApplicants": scoredApplicants}
json_outcome=json.dumps(output_dict)
json_outcome

'{"scoredApplicants": [{"name": "John", "score": 0.3}, {"name": "Jane", "score": 0.7}, {"name": "Gabe", "score": 0.7}, {"name": "Joe", "score": 0.5}]}'

In [ ]:
with open(scoredApplicants.json, 'w') as json_file:
    json_file.write(json_outcome)